In [1]:
import numpy as np
import pandas as pd

## I. Xử lý bộ dữ liệu

In [2]:
# Nhập đường dẫn của file dataset để chương trình đọc dữ liệu. (VD: wine.csv)
df = pd.read_csv(input(), sep=';')
df.squeeze()

 ..\Data\wine.csv


fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0               7.4             0.700         0.00             1.9      0.076   
1               7.8             0.880         0.00             2.6      0.098   
2               7.8             0.760         0.04             2.3      0.092   
3              11.2             0.280         0.56             1.9      0.075   
4               7.4             0.700         0.00             1.9      0.076   
...             ...               ...          ...             ...        ...   
1194            7.0             0.745         0.12             1.8      0.114   
1195            6.2             0.430         0.22             1.8      0.078   
1196            7.9             0.580         0.23             2.3      0.076   
1197            7.7             0.570         0.21             1.5      0.069   
1198            7.7             0.260         0.26             2.0      0.052   

      free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                    11.0                    34  0.99780  3.51       0.56   
1                    25.0                    67  0.99680  3.20       0.68   
2                    15.0                    54  0.99700  3.26       0.65   
3                    17.0                    60  0.99800  3.16       0.58   
4                    11.0                    34  0.99780  3.51       0.56   
...                   ...                   ...      ...   ...        ...   
1194                 15.0                    64  0.99588  3.22       0.59   
1195                 21.0                    56  0.99633  3.52       0.60   
1196                 23.0                    94  0.99686  3.21       0.58   
1197                  4.0                     9  0.99458  3.16       0.54   
1198                 19.0                    77  0.99510  3.15       0.79   

      alcohol  quality  
0         9.4        5  
1         9.8        5  
2         9.8        5  
3         9.8        6  
4         9.4        5  
...       ...      ...  
1194      9.5        6  
1195      9.5        6  
1196      9.5        6  
1197      9.8        6  
1198     10.9        6  

[1199 rows x 12 columns]

In [3]:
# Lấy danh sách các header của bộ dữ liệu.
headers = list(df.columns.values)

# Chuyển đổi bộ dữ liệu sang kiểu numpy array để dễ dàng thao tác.
data = np.array(df)

## II. Xây dựng mô hình đánh giá chất lượng rượu (Hồi quy tuyến tính)

### <mark>a. Sử dụng toàn bộ 11 đặc trưng đề bài cung cấp.</mark>

##### <u>Cài đặt các chức năng</u>

In [4]:
# Tính x mũ. (A @ x = b)
def calc_x_hat(data):
    A = data[:,:-1]
    b = data[:,-1:].reshape(len(data),)
    return np.linalg.pinv(A) @ b

In [5]:
# Kết quả của phương pháp hồi quy tuyến tính áp dụng trên 1 bộ dữ liệu.
def linear_regression(data, headers):
    x_hat = calc_x_hat(data)
        
    x_hat_data = []
    for i in range(len(x_hat)):
        row = []
        row.append('x'+ str(i + 1))
        row.append(headers[i])
        row.append(x_hat[i])
        x_hat_data.append(row)
    
    return pd.DataFrame(x_hat_data, columns=['x', 'Tính chất', 'Giá trị'])

##### <u>Kết quả</u>

In [6]:
# Kết quả của câu a.
linear_regression(data, headers)

x             Tính chất   Giá trị
0    x1         fixed acidity  0.005925
1    x2      volatile acidity -1.108038
2    x3           citric acid -0.263046
3    x4        residual sugar  0.015322
4    x5             chlorides -1.730503
5    x6   free sulfur dioxide  0.003801
6    x7  total sulfur dioxide -0.003899
7    x8               density  4.338588
8    x9                    pH -0.458535
9   x10             sulphates  0.729719
10  x11               alcohol  0.308859

#### <mark>b. Sử dụng duy nhất 1 đặc trưng cho kết quả tốt nhất. (Phương pháp Cross Validation)</mark>

##### <u>Cài đặt các chức năng</u>

In [7]:
# Tính sai số từ 1 cặp bộ dữ liệu train và test.
def calc_residual(train, test):
    x_hat = calc_x_hat(train)
    sum_residual = 0
    
    for i in range(len(test)):
        est_quality = 0
        for j in range(len(x_hat)):
            est_quality += x_hat[j] * test[i][j]
        sum_residual += np.abs(est_quality - test[i][-1])
        
    return sum_residual / len(test)

In [8]:
# Tính sai số trung bình của 1 bộ dữ liệu theo phương pháp Cross Validation.
def calc_avg_residual(data, model_num):
    n = int(np.round(len(data) / model_num))
    models = [data[i*n:i*n+n, :] for i in range(model_num-1)]
    models.append(data[(model_num-1)*n:, :])

    residuals = []
    for i in range(model_num):
        train = np.concatenate(models[:-1], axis=0)
        test = models[-1]
        residuals.append(calc_residual(train, test))
        
        temp = models[-1]
        models[-1] = models[i]
        models[i] = temp
    
    return np.average(residuals), residuals

In [9]:
# Tính các sai số trung bình của các bộ dữ liệu (mỗi bộ dữ liệu ứng với duy nhất 1 đặc trưng).
def calc_feature_avg_residuals(data, model_num):
    avg_residuals = []
    residuals = []
    
    for i in range(len(data[0]) - 1):
        feature_data = np.concatenate((data[:, i:i+1], data[:, -1:]), axis=1)
        feature_avg_residual, feature_residuals = calc_avg_residual(feature_data, model_num)
        avg_residuals.append(feature_avg_residual)
        residuals.append(feature_residuals)
    
    return avg_residuals, residuals

In [10]:
# Bảng xếp hạng so sánh sai số của mỗi bộ dữ liệu ứng với mỗi tính chất.
def ranking_table(data, headers):
    avg_residuals, residuals = calc_feature_avg_residuals(data, 4)
    indices = np.argsort(avg_residuals)
    
    ranking_data = []
    for i in range(len(indices)):
        row = []
        row.append(headers[indices[i]])
        row.append(residuals[indices[i]][0])
        row.append(residuals[indices[i]][1])
        row.append(residuals[indices[i]][2])
        row.append(residuals[indices[i]][3])
        row.append(avg_residuals[indices[i]])
        row.append(i + 1)
        ranking_data.append(row)
        
    return pd.DataFrame(ranking_data, columns=["Tính chất", "Sai số A", "Sai số B", "Sai số C", "Sai số D", "Sai số trung bình", "Xếp hạng"])

In [11]:
# Lọc bộ dữ liệu gốc chỉ còn 1 tính chất cho kết quả sai số bé nhất.
def build_best_data(data, headers):
    avg_residuals, _ = calc_feature_avg_residuals(data, 4)
    best_index = np.argsort(avg_residuals)[0]
    
    new_data = np.concatenate((data[:, best_index:best_index+1], data[:,-1:]), axis=1)
    new_headers = headers[best_index:best_index+1] + headers[-1:]
    
    return new_data, new_headers

##### <u>Kết quả</u>

In [12]:
# In bảng xếp hạng.
ranking_table(data, headers)

Tính chất  Sai số A  Sai số B  Sai số C  Sai số D  \
0                alcohol  0.556146  0.525760  0.596566  0.512212   
1                density  0.782727  0.696102  0.685753  0.694613   
2                     pH  0.791555  0.722449  0.736420  0.720151   
3          fixed acidity  1.221617  0.856488  1.239303  0.991560   
4              sulphates  1.200110  1.253365  0.935362  1.004242   
5       volatile acidity  2.296810  1.618290  1.841731  1.700559   
6         residual sugar  2.545658  1.988977  1.894290  1.628064   
7              chlorides  2.536739  1.932540  2.099208  1.869860   
8            citric acid  2.488746  2.948598  2.370226  2.786764   
9    free sulfur dioxide  3.255725  2.518466  3.123854  2.476097   
10  total sulfur dioxide  3.647015  2.798878  3.230203  2.880968   

    Sai số trung bình  Xếp hạng  
0            0.547671         1  
1            0.714799         2  
2            0.742644         3  
3            1.077242         4  
4            1.098270         5  
5            1.864348         6  
6            2.014247         7  
7            2.109587         8  
8            2.648583         9  
9            2.843535        10  
10           3.139266        11

In [13]:
# Kết quả của câu b.
new_data, new_headers = build_best_data(data, headers)
linear_regression(new_data, new_headers)

x Tính chất   Giá trị
0  x1   alcohol  0.543706

#### <mark>c. Xây dựng một mô hình của riêng bạn cho kết quả tốt nhất.</mark>

##### <u>Cài đặt các chức năng</u>

In [14]:
# Xây dựng lại bộ dữ liệu mới tốt hơn dựa trên bộ dữ liệu gốc.
# Thêm 1 tính chất mới là tổng của 2 tính chất tốt nhất được xếp hạng từ câu b.
def build_my_data(data, headers):
    n = 2
    avg_residuals, _ = calc_feature_avg_residuals(data, 4)
    indices = np.argsort(avg_residuals)[:n]
    
    new_data = np.array([data[:, i:i+1] for i in indices])
    new_data = new_data.sum(axis=0)
    new_data = np.concatenate((new_data, data), axis=1)
    
    new_headers = ['Tính chất mới'] + headers

    return new_data, new_headers

In [15]:
# So sánh sai số của mô hình câu a (11 tính chất) và sai số của mô hình mới (12 tính chất).
def compare(data, headers):
    avg_residual = calc_avg_residual(data, 4)[0]
    print('* ' + str(avg_residual) + ' là sai số của mô hình hồi quy tuyến tính với 11 tính chất.')
    
    new_data, _ = build_my_data(data, headers)
    new_avg_residual = calc_avg_residual(new_data, 4)[0]
    print('* ' + str(new_avg_residual) + ' là sai số của mô hình hồi quy tuyến tính mới.')
    
    delta = new_avg_residual - avg_residual
    print()
    print('Sai số của mô hình mới ', end='')
    if delta < 0:
        print('bé hơn', end='')
    elif delta > 0:
        print('lớn hơn', end='')
    else:
        print('bằng với', end='')
    print(' sai số của mô hình với 11 tính chất.')
    print('(' + str(np.abs(delta)) + ' là chênh lệch giữa 2 sai số này)')

##### <u>Kết quả</u>

In [16]:
# In ra kết quả câu c.
my_data, my_headers = build_my_data(data, headers)
linear_regression(my_data, my_headers)

x             Tính chất   Giá trị
0    x1         Tính chất mới  1.549149
1    x2         fixed acidity  0.005925
2    x3      volatile acidity -1.108038
3    x4           citric acid -0.263046
4    x5        residual sugar  0.015322
5    x6             chlorides -1.730503
6    x7   free sulfur dioxide  0.003801
7    x8  total sulfur dioxide -0.003899
8    x9               density  2.789439
9   x10                    pH -0.458535
10  x11             sulphates  0.729719
11  x12               alcohol -1.240290

In [17]:
# So sánh sai số của mô hình câu a (11 tính chất) và sai số của mô hình mới (12 tính chất).
compare(data, headers)

* 0.5150558048747267 là sai số của mô hình hồi quy tuyến tính với 11 tính chất.
* 0.5150558048747231 là sai số của mô hình hồi quy tuyến tính mới.

Sai số của mô hình mới bé hơn sai số của mô hình với 11 tính chất.
(3.552713678800501e-15 là chênh lệch giữa 2 sai số này)
